Rotational-vibrational solutions for H2S molecule.

Compute rovibrational energies, wavefunctions, and matrix elements for H2S, up to high rotational excitations ($J\sim 60$) at which rotational cluster states are formed.

In [1]:
import jax
jax.config.update("jax_platform_name", "cpu")
jax.config.update("jax_enable_x64", True)

import bz2
import gzip

import h5py
import numpy as np
from jax import numpy as jnp
from numpy.polynomial.hermite import hermgauss
from rovib import c2v
from rovib.h2s_dipole import batch_dipole
from rovib.h2s_potential import potential
from rovib.h2s_spinrot import spinrot_const_bisector
from rovib.keo import Molecule, batch_Gmat, batch_pseudo, com
from rovib.primbas import hermite
from rovib.symtop import rotme_cor, rotme_ovlp, rotme_rot
from rovib.vib_xy2 import vibrations_xy2
from scipy import optimize

## Vibrational coordinates and molecular frame

Define atomic masses and internal coordinates for describing vibrations. Here, we use the valence-bond coordinates, $r_1\equiv \text{S--H}_1$, $r_2\equiv \text{S--H}_2$, and $\alpha = \angle\text{H}_1\text{SH}_2$. The function `valence_bond_coordinates` is required to build the kinetic energy operator.

In [2]:
NCOO = 3 # number of vibrational coordinates
MASS_S = 31.97207070
MASS_H = 1.00782505

@com
def valence_bond_coordinates(coords):
    r1, r2, alpha = coords
    return jnp.array(
        [
            [0.0, 0.0, 0.0],
            [r1 * jnp.cos(alpha / 2), 0.0, r1 * jnp.sin(alpha / 2)],
            [r2 * jnp.cos(alpha / 2), 0.0, -r2 * jnp.sin(alpha / 2)],
        ]
    )

Molecule.masses = np.array([MASS_S, MASS_H, MASS_H])
Molecule.internal_to_cartesian = valence_bond_coordinates

We use Hermite functions $H_n(x)e^{-x^2/2}$ as the vibrational basis functions and employ Gauss-Hermite quadratures for computing matrix elements.

The Hermite functions are defined over the coordinate range $x\in(-\infty,\infty)$. To map $x$ into the vibrational valence bond coordinates $r_1\in(0,\infty)$, $r_2\in(0,\infty)$, and $\alpha=(0,\pi)$,
we use linear transformations $r_1=a_1x_1+b_1$, $r_2=a_2x_2+b_2$, $\alpha=a_3x_3+b_3$.
The parameters $a_1, b_1, ..., b_3$ are determined by mapping the vibrational Hamiltonian in valence-bond coordinates onto the harmonic oscillator Hamiltonian.

In [3]:
vmin = optimize.minimize(potential, [1.0, 1.0, np.pi / 2])
r0 = vmin.x
v0 = vmin.fun
print("mininum of the potential:", r0, v0)

print("Cartesian coordinates of atoms:", valence_bond_coordinates(r0))

freq = jnp.diag(jax.hessian(potential)(r0))
mu = jnp.diag(batch_Gmat(jnp.array([r0]))[0, :NCOO, :NCOO])
lin_a = jnp.sqrt(jnp.sqrt(mu / freq))
lin_b = r0

print("x->r linear mapping parameters 'a' and 'b':", lin_a, lin_b)

# x->r linear mapping function
x_to_r_map = lambda x: lin_a * x + lin_b

mininum of the potential: [1.3358387  1.3358387  1.61042427] -0.0007846164036985615
Cartesian coordinates of atoms: [[-5.48977100e-02  0.00000000e+00  1.12725516e-12]
 [ 8.70782813e-01  0.00000000e+00  9.63109860e-01]
 [ 8.70782813e-01  0.00000000e+00 -9.63109860e-01]]
x->r linear mapping parameters 'a' and 'b': [0.11245619 0.11245619 0.17845517] [1.3358387  1.3358387  1.61042427]


We will be computing rovibrational matrix elements of spin-rotation tensors, which were obtained in a quantum chemical calculation in some reference coordinate frame. This frame may be different from the one used here as defined by the `valence_bond_coordinates` function. Therefore, the spin-rotation tensors need to be rotated to the same coordinate frame as used here for rovibrational solutions.

Such frame transformation is already coded in the module `rovib.h2s_spinrot` and here we check that Cartesian coordinates of atoms after the transformation match those returned by the `valence_bond_coordinates` function.

In [7]:
from rovib.h2s_spinrot import _COORDS

print("Reference internal coordiantes:", _COORDS)

sr, atom_xyz0 = spinrot_const_bisector(np.array([_COORDS]), return_atom_xyz=True)
sr1 = sr[:, 0]
sr2 = sr[:, 1]
print("Reference Cartesian coordinates of atoms:\n", atom_xyz0)

atom_xyz = valence_bond_coordinates(_COORDS)
print("Calculated from internal Cartesian coordinates of atoms:\n", atom_xyz)

print("max difference:", np.max(np.abs(atom_xyz0 - atom_xyz), axis=-1))

Reference internal coordiantes: [1.35, 1.35, 1.605702911834783]
Reference Cartesian coordinates of atoms:
 [[-0.05561581  0.          0.        ]
 [ 0.88217319  0.          0.97110893]
 [ 0.88217319  0.         -0.97110893]]
Calculated from internal Cartesian coordinates of atoms:
 [[-5.56157943e-02  0.00000000e+00 -1.41018406e-18]
 [ 8.82173006e-01  0.00000000e+00  9.71108730e-01]
 [ 8.82173006e-01  0.00000000e+00 -9.71108730e-01]]
max difference: [1.08848569e-08 2.02253066e-07 2.02253066e-07]


The electric dipole moment surface is implemented in `h2s_dipole.batch_dipole`, which takes the molecule's internal coordinates and a function for transforming internal to Cartesian coordinates as parameters. Define a new wrapper function, `dipole_func`, that automatically uses the `valence_bond_coordinates` function, as defined above, for the coordinate transformation.

In [8]:
dipole_func = lambda q: batch_dipole(q, valence_bond_coordinates)

## Vibrational basis set contraction

To optimize the vibrational basis set, we begin by solving a simplified vibrational problem for each individual vibrational coordinate.
For this, we set the basis functions for all coordinates to $e^{−x^2/2}$ except for one.
We then use the corresponding solutions obtained sequentially for each coordinate to construct a contracted product basis set.

In [9]:
# list of 1D primitive basis functions for each vibrational coordinate
list_psi = (hermite, hermite, hermite)

# number of primitive basis functions for each vibrational coordinate
nmax = 60

# number of quadrature points for each vibrational coordinate
npoints = 80

contr_vec = []

for icoo in range(NCOO):
    print(f"\nSolve for coordinate #{icoo}")

    # quanta, set quanta to zero for all coordinates except the `icoo`
    list_q = [np.arange(nmax) if i == icoo else [0] for i in range(NCOO)]

    # quadratures, reduce number of points for all coordinates except the `icoo`
    n = [npoints if i == icoo else 10 for i in range(NCOO)]
    n1, n2, n3 = n
    x1, w1 = hermgauss(n1)
    x2, w2 = hermgauss(n2)
    x3, w3 = hermgauss(n3)
    w1 /= np.exp(-(x1**2))
    w2 /= np.exp(-(x2**2))
    w3 /= np.exp(-(x3**2))
    list_x = (x1, x2, x3)
    list_w = (w1, w2, w3)

    # solve for basis functions for the `icoo` vibrational coordinate
    e, v, *_ = vibrations_xy2(
        list_psi,
        list_x,
        list_w,
        list_q,
        select_points=lambda x, w: True,
        select_quanta=lambda q: np.sum(q * np.array([1, 1, 1])) <= nmax,
        x_to_r_map=x_to_r_map,
        gmat=lambda x: batch_Gmat(x),
        pseudo=batch_pseudo,
        potential=potential,
    )

    # keep eigenvectors
    contr_vec.append(v)

    print(f"energies:")
    print(e[0], e - e[0])


Solve for coordinate #0
energies:
3337.3945251424348 [0.00000000e+00 2.62278235e+03 5.15020514e+03 7.58246436e+03
 9.91956242e+03 1.21613549e+04 1.43075887e+04 1.63579336e+04
 1.83120078e+04 2.01694004e+04 2.19297332e+04 2.35935476e+04
 2.51710003e+04 2.67148155e+04 2.83313983e+04 3.00991224e+04
 3.20328613e+04 3.41237612e+04 3.63621854e+04 3.87414444e+04
 4.12573394e+04 4.39074679e+04 4.66907551e+04 4.96071642e+04
 5.26575238e+04 5.58434264e+04 5.91671798e+04 6.26317880e+04
 6.62409718e+04 6.99992056e+04 7.39117769e+04 7.79849510e+04
 8.22259096e+04 8.66433924e+04 9.12472636e+04 9.60490463e+04
 1.01065534e+05 1.06306725e+05 1.11805114e+05 1.17593906e+05
 1.23605946e+05 1.30225655e+05 1.37034951e+05 1.43667742e+05
 1.55286600e+05 1.79306940e+05 2.21355661e+05 2.91280671e+05
 4.07970417e+05 6.06459296e+05 9.51882898e+05 1.56686201e+06
 2.68649466e+06 4.77259926e+06 8.76173508e+06 1.66359274e+07
 3.28451650e+07 6.82888840e+07 1.53726954e+08 4.04395898e+08]

Solve for coordinate #1
energ

Transform primitive basis

In [10]:
# override primitive basis with contracted basis
list_psi = [
    lambda x, n: jnp.dot(hermite(x, n), contr_vec[0]),
    lambda x, n: jnp.dot(hermite(x, n), contr_vec[1]),
    lambda x, n: jnp.dot(hermite(x, n), contr_vec[2]),
]

## Vibrational energies and matrix elements

Using the optimized basis set from the previous step, we solve the vibrational problem employing a polyad basis set truncation.
In this approach, we include only those products of functions in the basis set for which the sum of the corresponding quanta, weighted by specific factors,
is less than or equal to a certain maximum value.
Specifically, we include basis functions that satisfy the condition $2n_1+2n_2+n_3\leq P_\text{max}$, where $n_1$, $n_2$, and $n_3$ are the quantum numbers for the two stretching and one bending coordinates, respectively.

In [11]:
# polyad number for basis set truncation
pmax = 24

# quanta
list_q = [np.arange(nmax) for i in range(NCOO)]

# quadratures
n = [npoints for i in range(NCOO)]
n1, n2, n3 = n
x1, w1 = hermgauss(n1)
x2, w2 = hermgauss(n2)
x3, w3 = hermgauss(n3)
w1 /= np.exp(-(x1**2))
w2 /= np.exp(-(x2**2))
w3 /= np.exp(-(x3**2))
list_x = (x1, x2, x3)
list_w = (w1, w2, w3)

# solutions, including vibrational matrix elements of the rotational part of kinetic energy operator
#   as well as matrix elements of spin-rotation tensors and dipole moment operator
vib_enr, vib_vec, vib_sym, vib_quanta, grot_me, gcor_me, ext_oper_me = vibrations_xy2(
    list_psi,
    list_x,
    list_w,
    list_q,
    select_points=lambda x, w: True,
    select_quanta=lambda q: np.sum(q * np.array([2, 2, 1])) <= pmax,
    x_to_r_map=x_to_r_map,
    gmat=lambda x: batch_Gmat(x),
    pseudo=batch_pseudo,
    potential=potential,
    assign_c2v=True,  # assign symmetries to vibrational states
    ext_oper_list=[
        spinrot_const_bisector,
        dipole_func,
        lambda q: q,
    ],  # spin-rotation tensor, dipole moment, and coordinate operators
)

Print vibrational energies with asssignments

In [12]:
nbas_vib = vib_vec.shape[-1]
print("number of vibrational states:", nbas_vib)

ind = np.argmax(np.abs(vib_vec), axis=0)
# add vibrational state symmetry to the vibrational quanta assignment
vib_quanta_j0 = np.concatenate((vib_quanta[ind], np.array(vib_sym)[:, None]), axis=1)
zpe = vib_enr[0]

for e, q in zip(vib_enr, vib_quanta_j0):
    print(e - zpe, q)

number of vibrational states: 819
0.0 ['0' '0' '0' 'A1']
1182.5695784823479 ['0' '0' '1' 'A1']
2353.9072471024006 ['0' '0' '2' 'A1']
2614.3947685784237 ['0' '1' '0' 'A1']
2628.463252259177 ['1' '0' '0' 'B2']
3513.7049677656955 ['0' '0' '3' 'A1']
3779.189281127134 ['1' '0' '1' 'A1']
3789.269797155733 ['0' '1' '1' 'B2']
4661.605684865226 ['0' '0' '4' 'A1']
4932.688955308939 ['0' '1' '2' 'A1']
4939.1298367897125 ['1' '0' '2' 'B2']
5145.031968457824 ['2' '0' '0' 'A1']
5147.166700473794 ['0' '2' '0' 'B2']
5243.1588958393495 ['1' '1' '0' 'A1']
5797.207416810754 ['0' '0' '5' 'A1']
6074.566096236209 ['1' '0' '3' 'A1']
6077.62662872564 ['0' '1' '3' 'B2']
6288.1350643886235 ['2' '0' '1' 'A1']
6289.128599534664 ['0' '2' '1' 'B2']
6385.320941966094 ['1' '1' '1' 'A1']
6920.081232847879 ['0' '0' '6' 'A1']
7204.308539640222 ['1' '0' '4' 'B2']
7204.435493958041 ['0' '1' '4' 'A1']
7419.848918299833 ['2' '0' '2' 'A1']
7420.078936606571 ['0' '2' '2' 'B2']
7516.826366763387 ['1' '1' '2' 'A1']
7576.4143612

Store vibrational matrix elements of spin-rotation tensors, dipole moment, and coordinate operators in file. Continue to compute rovibrational solutions for $J=0..J_\text{max}$ and store them in files. The rovibrational matrix elements of spin-rotation tensors and dipole moment will be computed in `h2s_cart_me.ipynb`.

In [13]:

sr_h1, sr_h2 = np.moveaxis(ext_oper_me[0], 2, 0)
dip = ext_oper_me[1]
coo = ext_oper_me[2]

h5 = h5py.File(f"h2s_vibme_pmax{pmax}.h5", "w")
h5_sr = h5.create_group("spin-rotation")
h5_sr.create_dataset("h1", data=sr_h1)
h5_sr.create_dataset("h2", data=sr_h2)
h5.create_dataset("dipole-moment", data=dip)
h5.create_dataset("coordinate", data=coo)
h5.close()

## Rotational basis and matrix elements

For a selected $J$ quantum number of the rotational angular momentum, compute matrix elements $\lang J',k',\tau'|J,k,\tau \rang$, $\lang J',k',\tau'|\hat{J}_\alpha\hat{J}_\beta|J,k,\tau \rang$, and $\lang J',k',\tau'|\hat{J}_\alpha|J,k,\tau \rang$ ($\alpha,\beta=x,y,z$). Where $|J,k,\tau\rang$ are symmetric-top functions in Wang's (symmetry-adapted) representation, $k=0..J$ and $\tau=0,1$ (parity).

In [14]:
j_angmom = 10

# matrix elements <jk'|jk>
s_rot, _, jktau_quanta = rotme_ovlp(j_angmom)

# matrix elements <jk'|Ja*Jb|jk>
jab_rot, *_ = rotme_rot(j_angmom)

# matrix elements <jk'|i*Ja|jk>
ja_rot, *_ = rotme_cor(j_angmom)

nbas_rot = len(jktau_quanta)

print("rotational angular momentum:", j_angmom)
print("number of rotational states:", nbas_rot)

rotational angular momentum: 10
number of rotational states: 21


## Symmetry-adapted rovibrational basis and solutions

In [15]:
# combine vibrational and rotational quanta [(v1, v2, v3, vib_sym, j, k, tau, rot_sym), ...]
rovib_quanta = np.concatenate(
    (
        vib_quanta_j0[:, None, :].repeat(nbas_rot, axis=1),
        jktau_quanta[None, :, :].repeat(nbas_vib, axis=0),
    ),
    axis=-1,
).reshape(-1, 8)

# mapping between rovibrational state index and indices of vibrational and rotational functions
rovib_ind = np.concatenate(
    (
        np.arange(nbas_vib)[:, None, None].repeat(nbas_rot, axis=1),
        np.arange(nbas_rot)[None, :, None].repeat(nbas_vib, axis=0),
    ),
    axis=-1,
).reshape(-1, 2)

# symmetry of rovibrational product prod_sym = vib_sym * rot_sym
prod_sym = np.array(
    [c2v.C2V_PRODUCT_TABLE[(sym1, sym2)] for (sym1, sym2) in rovib_quanta[:, (3, 7)]]
)
# update rovibrational quanta [(j, prod_sym, v1, v2, v3, vib_sym, k, tau, rot_sym), ...]
rovib_quanta = np.concatenate(
    (prod_sym[:, None], rovib_quanta[:, (4, 0, 1, 2, 3, 5, 6, 7)]), axis=-1
)

# identify indices of rovibrational product states for each symmetry
ind_sym = {sym: np.where(rovib_quanta[:, 0] == sym)[0] for sym in c2v.C2V_IRREPS}

# ... indices of vibrational and rotational product states for each symmetry
rovib_ind_sym = {sym: rovib_ind[ind] for sym, ind in ind_sym.items()}

Compute and diagonalise total Hamiltonian matrix for different symmetries

In [16]:
for sym, ind in ind_sym.items():
    vind, rind = rovib_ind[ind].T

    rot_me = jnp.einsum(
        "ijab,ijab->ij", grot_me[np.ix_(vind, vind)], jab_rot[np.ix_(rind, rind)]
    )
    cor_me = jnp.einsum(
        "ija,ija->ij", gcor_me[np.ix_(vind, vind)], ja_rot[np.ix_(rind, rind)]
    )
    vib_me = jnp.diag(vib_enr[vind]) * s_rot[np.ix_(rind, rind)]
    hmat = vib_me + 0.5 * (rot_me + cor_me)

    enr, vec = jnp.linalg.eigh(hmat)
    qua = rovib_quanta[ind[jnp.argmax(vec, axis=0)]]
    for e, q in zip(enr[:20], qua[:20]):
        print(e-zpe, q)

568.4784997362926 ['A1' '10' '0' '0' '0' 'A1' '0' '0' 'A1']
742.8545565005593 ['A1' '10' '0' '0' '0' 'A1' '4' '0' 'A1']
875.6311638284728 ['A1' '10' '0' '0' '0' 'A1' '8' '0' 'A1']
960.9040137031911 ['A1' '10' '0' '0' '0' 'A1' '8' '0' 'A1']
1013.3355752999028 ['A1' '10' '0' '0' '0' 'A1' '8' '0' 'A1']
1094.4831305240718 ['A1' '10' '1' '0' '0' 'B2' '9' '0' 'B2']
1747.7348010113433 ['A1' '10' '0' '0' '1' 'A1' '0' '0' 'A1']
1933.9995826928148 ['A1' '10' '0' '0' '1' 'A1' '6' '0' 'A1']
2075.2228451536735 ['A1' '10' '0' '0' '1' 'A1' '6' '0' 'A1']
2164.8682857938898 ['A1' '10' '0' '0' '1' 'A1' '2' '0' 'A1']
2222.0028632359977 ['A1' '10' '0' '0' '1' 'A1' '10' '0' 'A1']
2310.0480979962126 ['A1' '10' '0' '0' '0' 'A1' '10' '0' 'A1']
2916.006363394627 ['A1' '10' '0' '0' '2' 'A1' '0' '0' 'A1']
3114.859281040652 ['A1' '10' '0' '0' '2' 'A1' '6' '0' 'A1']
3174.4069131961746 ['A1' '10' '0' '1' '0' 'A1' '0' '0' 'A1']
3190.1732035313003 ['A1' '10' '1' '0' '0' 'B2' '1' '0' 'B2']
3264.8890035793033 ['A1' '10

## Run rovibrational calculations for $J=0..J_\text{max}$

Put three previous cells into one and add a loop over $J$.

In [19]:
min_j = 0
max_j = 60
max_energy = 40000

# store energies below `max_energy` in text file
fl_enr = gzip.open(f"h2s_energies_pmax{pmax}_jmax{max_j}.txt.gz", "wt")

for j_angmom in range(min_j, max_j + 1):

    # store energies and wavefunctions in hdf5 file
    h5 = h5py.File(f"h2s_coefficients_pmax{pmax}_j{j_angmom}.h5", "w")
    h5_enr = h5.create_group("energies")
    h5_coef = h5.create_group("coefficients")
    h5_vind = h5.create_group("vib-indices")
    h5_rind = h5.create_group("rot-indices")
    h5_qua = h5.create_group("quanta")

    s_rot, _, ktau_quanta = rotme_ovlp(j_angmom)
    jab_rot, *_ = rotme_rot(j_angmom)
    ja_rot, *_ = rotme_cor(j_angmom)
    nbas_rot = len(ktau_quanta)

    print("solve for rotational angular momentum:", j_angmom)

    rovib_quanta = np.concatenate(
        (
            vib_quanta_j0[:, None, :].repeat(nbas_rot, axis=1),
            ktau_quanta[None, :, :].repeat(nbas_vib, axis=0),
        ),
        axis=-1,
    ).reshape(-1, 8)

    rovib_ind = np.concatenate(
        (
            np.arange(nbas_vib)[:, None, None].repeat(nbas_rot, axis=1),
            np.arange(nbas_rot)[None, :, None].repeat(nbas_vib, axis=0),
        ),
        axis=-1,
    ).reshape(-1, 2)

    prod_sym = np.array(
        [
            c2v.C2V_PRODUCT_TABLE[(sym1, sym2)]
            for (sym1, sym2) in rovib_quanta[:, (3, 7)]
        ]
    )
    rovib_quanta = np.concatenate(
        (prod_sym[:, None], rovib_quanta[:, (4, 0, 1, 2, 3, 5, 6, 7)]), axis=-1
    )

    ind_sym = {sym: np.where(rovib_quanta[:, 0] == sym)[0] for sym in c2v.C2V_IRREPS}
    # rovib_ind_sym = {sym: rovib_ind[ind] for sym, ind in ind_sym.items()}

    for sym, ind in ind_sym.items():
        vind, rind = rovib_ind[ind].T
        qua = rovib_quanta[ind]

        rot_me = jnp.einsum(
            "ijab,ijab->ij", grot_me[np.ix_(vind, vind)], jab_rot[np.ix_(rind, rind)]
        )
        cor_me = jnp.einsum(
            "ija,ija->ij", gcor_me[np.ix_(vind, vind)], ja_rot[np.ix_(rind, rind)]
        )
        vib_me = jnp.diag(vib_enr[vind]) * s_rot[np.ix_(rind, rind)]
        hmat = vib_me + 0.5 * (rot_me + cor_me)
        enr, vec = jnp.linalg.eigh(hmat)

        if len(vec) == 0:  # no A2 and B1 states for J=0
            continue

        # store data in hdf5 file
        enr_ind = np.where(enr - zpe < max_energy)[0]
        h5_enr.create_dataset(sym, data=enr[enr_ind])
        h5_coef.create_dataset(sym, data=vec[:, enr_ind])
        h5_vind.create_dataset(sym, data=vind)
        h5_rind.create_dataset(sym, data=rind)
        qua_str = [",".join(elem) for elem in qua]
        max_len = max([len(elem) for elem in qua_str])
        qua_ascii = [elem.encode("ascii", "ignore") for elem in qua_str]
        h5_qua.create_dataset(sym, (len(qua_ascii), 1), f"S{max_len}", data=qua_ascii)

        qua = rovib_quanta[ind[jnp.argmax(vec, axis=0)]]

        # store energies and assignments in text file
        for e, q in zip(enr, qua):
            e_ = e - zpe
            if e_ < max_energy:
                fl_enr.write(
                    f"{e_:18.6f}    " + " ".join(f"{elem:4s}" for elem in q) + "\n"
                )

    h5.close()

fl_enr.close()

solve for rotational angular momentum: 0
solve for rotational angular momentum: 1
solve for rotational angular momentum: 2
solve for rotational angular momentum: 3
solve for rotational angular momentum: 4
solve for rotational angular momentum: 5
solve for rotational angular momentum: 6
solve for rotational angular momentum: 7
solve for rotational angular momentum: 8
solve for rotational angular momentum: 9
solve for rotational angular momentum: 10
solve for rotational angular momentum: 11
solve for rotational angular momentum: 12
solve for rotational angular momentum: 13
solve for rotational angular momentum: 14
solve for rotational angular momentum: 15
solve for rotational angular momentum: 16
solve for rotational angular momentum: 17
solve for rotational angular momentum: 18
solve for rotational angular momentum: 19
solve for rotational angular momentum: 20
solve for rotational angular momentum: 21
solve for rotational angular momentum: 22
solve for rotational angular momentum: 23
so

Compare calculated rovibrational energies with results of DVR3D calculations, published in [Azzam, A. A. A., Tennyson, J., Yurchenko, S. N., Naumenko, O. V., "ExoMol molecular line lists – XVI: The rotation-vibration spectrum of hot H2S", Monthly Notices of the Royal Astronomical Society 460, 4063-4074 (2016)](https://doi.org/10.1093/mnras/stw1133). The reference data is stored in file 'etc/benchmark/1H2-32S__AYT2_20191004.states.bz2', for further details, see the 'etc/benchmark/README.md'.

In [20]:
max_energy = 10000
max_j = 20

# read reference energies
ref_enr = {}
with bz2.open("etc/benchmark/1H2-32S__AYT2_20191004.states.bz2", "rt") as fl:
    for line in fl:
        w = line.split()
        e = float(w[1])
        j = int(w[3])
        sym = w[5]
        if e > max_energy or j > max_j:
            continue
        try:
            ref_enr[(j, sym)].append(e)
        except KeyError:
            ref_enr[(j, sym)] = [e]
ref_enr = {j: sorted(e) for j, e in ref_enr.items()}

# read calculated energies
cal_enr = {}
with gzip.open(f"h2s_energies_pmax{pmax}_jmax60.txt.gz", "rt") as fl:
    for line in fl:
        w = line.split()
        e = float(w[0])
        j = int(w[2])
        sym = w[1]
        if e > max_energy or j > max_j:
            continue
        try:
            cal_enr[(j, sym)].append(e)
        except KeyError:
            cal_enr[(j, sym)] = [e]
cal_enr = {j: sorted(e) for j, e in cal_enr.items()}

print("max and mean differences between the calculated and reference energies")
for j, sym in cal_enr.keys():
    diff = np.array(cal_enr[(j, sym)]) - np.array(ref_enr[(j, sym)])
    max_diff = np.max(np.abs(diff))
    mean_diff = np.mean(np.abs(diff))
    print(f"j = {j}  sym = {sym}  no_states = {len(diff)}  max_diff = {max_diff}  mean_diff = {mean_diff}")

max and mean differences between the calculated and reference energies
j = 0  sym = A1  no_states = 33  max_diff = 0.006169999998746789  mean_diff = 0.0009063939393826965
j = 0  sym = B2  no_states = 18  max_diff = 0.006396000000677304  mean_diff = 0.001302555555589101
j = 1  sym = A1  no_states = 18  max_diff = 0.006344999999782885  mean_diff = 0.001291166666861601
j = 1  sym = A2  no_states = 50  max_diff = 0.006344000001263339  mean_diff = 0.0010005200001343083
j = 1  sym = B1  no_states = 50  max_diff = 0.006343000000924803  mean_diff = 0.0010008200001266232
j = 1  sym = B2  no_states = 32  max_diff = 0.0061189999996713595  mean_diff = 0.0008322500000411681
j = 2  sym = A1  no_states = 82  max_diff = 0.006242999999813037  mean_diff = 0.000924524390243229
j = 2  sym = A2  no_states = 50  max_diff = 0.006252000001040869  mean_diff = 0.0009842600000207825
j = 2  sym = B1  no_states = 50  max_diff = 0.006242999999813037  mean_diff = 0.0009827800000132925
j = 2  sym = B2  no_states = 68